<a href="https://colab.research.google.com/github/SuhailAl/BlockChainManager/blob/main/SLFTUTORIAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, accuracy_score, confusion_matrix, classification_report, roc_auc_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor


In [2]:
from google.colab import files
uploaded = files.upload()

Saving PRESCRIPTIONS.csv to PRESCRIPTIONS.csv


In [3]:
from google.colab import files
uploaded1 = files.upload()

Saving ADMISSIONS.csv to ADMISSIONS.csv


In [4]:
from google.colab import files
uploaded2 = files.upload()

Saving PATIENTS.csv to PATIENTS.csv


In [5]:
from google.colab import files
uploaded3 = files.upload()

Saving DIAGNOSES_ICD.csv to DIAGNOSES_ICD.csv


In [6]:
admissions = pd.read_csv('ADMISSIONS.csv').columns.str.lower()
diagnoses_icd = pd.read_csv('DIAGNOSES_ICD.csv').columns.str.lower()
patients = pd.read_csv('PATIENTS.csv').columns.str.lower()
prescriptions = pd.read_csv('PRESCRIPTIONS.csv').columns.str.lower()


In [10]:
admissions = pd.read_csv('ADMISSIONS.csv').columns.str.lower()
diagnoses_icd = pd.read_csv('DIAGNOSES_ICD.csv').columns.str.lower()
patients = pd.read_csv('PATIENTS.csv').columns.str.lower()
prescriptions = pd.read_csv('PRESCRIPTIONS.csv').columns.str.lower()


In [14]:
admissions.columns = admissions.columns.str.lower()
diagnoses_icd.columns = diagnoses_icd.columns.str.lower()
patients.columns = patients.columns.str.lower()
prescriptions.columns = prescriptions.columns.str.lower()

In [15]:
# Merge and prepare data
data_merged = pd.merge(patients[['subject_id', 'dob', 'gender']], admissions, on='subject_id', how='inner')
data_merged['admittime'] = pd.to_datetime(data_merged['admittime'])
data_merged['dischtime'] = pd.to_datetime(data_merged['dischtime'])
data_merged['dob'] = pd.to_datetime(data_merged['dob'])
data_merged['age_at_admission'] = data_merged['admittime'].dt.year - data_merged['dob'].dt.year
data_merged['los'] = (data_merged['dischtime'] - data_merged['admittime']).dt.days
data_merged['gender'] = data_merged['gender'].map({'M': 1, 'F': 0}).fillna(-1)


In [16]:
# Calculate days until next admission and create target variable for readmission within 30 days
data_sorted = data_merged.sort_values(by=['subject_id', 'admittime'])
data_sorted['next_admission_time'] = data_sorted.groupby('subject_id')['admittime'].shift(-1)
data_sorted['days_until_next_adm'] = (data_sorted['next_admission_time'] - data_sorted['dischtime']).dt.days
data_sorted['readmission_30'] = (data_sorted['days_until_next_adm'] <= 30) & (data_sorted['days_until_next_adm'] >= 0)

In [17]:
# Predicting readmission with Logistic Regression
X = data_sorted[['age_at_admission', 'los', 'gender']].fillna(0)
y = data_sorted['readmission_30']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9615384615384616
Classification Report:
               precision    recall  f1-score   support

       False       0.96      1.00      0.98        25
        True       0.00      0.00      0.00         1

    accuracy                           0.96        26
   macro avg       0.48      0.50      0.49        26
weighted avg       0.92      0.96      0.94        26



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
# Predicting Length of Stay for next admission within 30 days using various regressors
data_sorted['next_los'] = data_sorted.groupby('subject_id')['los'].shift(-1).where(data_sorted['readmission_30'])
data_for_severity = data_sorted.dropna(subset=['next_los'])
X_severity = data_for_severity[['age_at_admission', 'los', 'gender']]
y_severity = data_for_severity['next_los']
scaler = StandardScaler()
X_severity_scaled = scaler.fit_transform(X_severity)
X_severity_train, X_severity_test, y_severity_train, y_severity_test = train_test_split(X_severity_scaled, y_severity, test_size=0.2, random_state=42)
severity_models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100),
    "MLP Regressor": MLPRegressor(hidden_layer_sizes=(10,), max_iter=1000)
}
for name, model in severity_models.items():
    model.fit(X_severity_train, y_severity_train)
    y_severity_pred = model.predict(X_severity_test)
    mse = mean_squared_error(y_severity_test, y_severity_pred)
    print(f"{name} Mean Squared Error: {mse}")

Linear Regression Mean Squared Error: 31.306445478524115
Random Forest Mean Squared Error: 25.287633333333332
Gradient Boosting Mean Squared Error: 35.80421479795075
MLP Regressor Mean Squared Error: 15.101294585100165


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
